In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/home/jstonge/Documents/phd/side_quest/shambolics/posts/mongoDB_opt':
  os.chdir(r'/home/jstonge/Documents/phd/side_quest/shambolics/posts/mongoDB_opt')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


/tmp/ipykernel_36856/1083933970.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"/home/jstonge/miniconda3/envs/classify-comp-proj/lib/python3.9/importlib/_bootstrap.py": 1669300927.0, "/home/jstonge/miniconda3/envs/classify-comp-proj/lib/python3.9/importlib/_bootstrap_external.py": 1669300927.0, "/home/jstonge/miniconda3/envs/classify-comp-proj/lib/python3.9/codecs.py": 1669300926.0, "/home/jstonge/miniconda3/envs/classify-comp-proj/lib/python3.9/encodings/aliases.py": 1669300927.0, "/home/jstonge/miniconda3/envs/classify-comp-proj/lib/python3.9/encodings/__init__.py": 1669300927.0, "/home/jstonge/miniconda3/envs/classify-comp-proj/lib/python3.9/encodings/utf_8.py": 1669300927.0, "/home/jstonge/miniconda3/envs/classify-comp-proj/lib/python3.9/encodings/latin_1.py": 1669300927.0, "/home/jstonge/miniconda3/envs/classify-comp-proj/lib/python3.9/abc.py": 1669300926.0, "/home/jstonge/miniconda3/envs/classify-comp-proj/lib/python3.9/io.py": 1669300926.0, "/home/jstonge/miniconda3/envs/classify-comp-proj/lib/python3.9/stat.py": 1669300926.0, "/home/jstonge/miniconda3/en

In [2]:
#| echo: false 
from creds import client
from bson.objectid import ObjectId
from pymongo import ASCENDING, UpdateOne
from pprint import pprint
import time
db = client['papersDB']

In [3]:
#| echo: false
#| eval: false
def reinitialize_test_db():
    [db.paper_test.drop_index(i) for i in db.paper_test.index_information() if i != "_id_"]
    [db.works_oa_test.drop_index(i) for i in db.works_oa_test.index_information() if i != "_id_"]
    db.works_oa_test.update_many({}, { '$unset': { 'mag': '' } })
    db.paper_test.update_many({}, { '$unset': { 'doi': '' } })

reinitialize_test_db()

In [4]:
# Step in the aggregation pipeline that we'll use often
return_top_5 = { "$limit": 5 }
only_keep_existing_matches = { "$match": { "matches": { "$ne": [] } } }

In [5]:
#| eval: false
#| code-fold: true
db.works_oa_test.insert_many( list(db.works_oa.find(
    { "publication_year": 1960 },
    { "concepts": 1, "publication_date": 1, "doi": 1, "ids": 1 }
    ))
)

db.paper_test.insert_many( list(db.papers.find(
    { "year": 1960, "s2fieldsofstudy": { "$type": "array" } },
    { "externalids": 1, "s2fieldsofstudy": 1 }
    ))
)

In [6]:
print(f"S2orc has {db.paper_test.count_documents({})} papers with `DOI` in 1960")
print(f"OpenAlex has {db.works_oa_test.count_documents({})} papers with `DOI` in 1960")

S2orc has 408248 papers with `DOI` in 1960


OpenAlex has 478673 papers with `DOI` in 1960


In [7]:
pprint(db.paper_test.find_one({ "externalids.DOI": { "$type": "string" } }))

{'_id': ObjectId('63e50688a64b9c3ca0a7a028'),
 'doi': 'https://doi.org/10.3130/AIJSAXX.66.2.0_73',
 'externalids': {'ACL': None,
                 'ArXiv': None,
                 'CorpusId': '125229466',
                 'DBLP': None,
                 'DOI': '10.3130/AIJSAXX.66.2.0_73',
                 'MAG': '2752866858',
                 'PubMed': None,
                 'PubMedCentral': None},
 's2fieldsofstudy': [{'category': 'Engineering', 'source': 's2-fos-model'},
                     {'category': 'Geology', 'source': 'external'}]}


In [8]:
pprint(db.works_oa_test.find_one())

{'_id': ObjectId('642044928238317efbef9371'),
 'authorships': [],
 'concepts': [{'display_name': 'Political science',
               'id': 'https://openalex.org/C17744445',
               'level': 0,
               'score': 0.39196813,
               'wikidata': 'https://www.wikidata.org/wiki/Q36442'}],
 'doi': 'https://doi.org/10.6028/nbs.mp.237',
 'ids': {'doi': 'https://doi.org/10.6028/nbs.mp.237',
         'openalex': 'https://openalex.org/W4232940716'},
 'mag': None,
 'publication_date': '1960-01-01'}


In [9]:
#| code-fold: true
start_time = time.time()
db.paper_test.update_many(
    {},
    [
        { "$set": { 
            "doi": { 
                "$cond": [
                    { "$ne": ["$externalids.DOI", None] },
                    { "$concat": ["https://doi.org/", "$externalids.DOI"] },
                    None
                ]
                }
            } }
    ]
)
print("Setting a field on ~400K took --- %s seconds ---" % (time.time() - start_time))

Setting a field on ~400K took --- 1.1609840393066406 seconds ---


In [10]:
#| eval: false
start_time = time.time()

s2orc_doi_is_string = { "$match": { "doi": { "$type": "string" }} }

s2orc_to_oa_doi_lookup ={
      "$lookup": {
         "from": "works_oa_test",
         "localField": "doi",
         "foreignField": "doi",
         "as": "matches"
      }
   }

pipeline = [
   s2orc_doi_is_string,
   s2orc_to_oa_doi_lookup,
   only_keep_existing_matches,
]

assert 'doi' in db.paper_test.find_one().keys(), 'missing field in paper test'
assert 'doi' in db.works_oa_test.find_one().keys(), 'missing field in oa'

res = list(db.paper_test.aggregate( pipeline ))
print("Process finished --- %s seconds ---" % (time.time() - start_time))

In [11]:
db.works_oa_test.create_index([("doi", ASCENDING)])
db.paper_test.create_index([("doi", ASCENDING)])

'doi_1'

In [12]:
start_time = time.time()

s2orc_doi_is_string = { "$match": { "doi": { "$type": "string" }} }

s2orc_to_oa_doi_lookup ={
      "$lookup": {
         "from": "works_oa_test",
         "localField": "doi",
         "foreignField": "doi",
         "as": "matches"
      }
   }

res = list(db.paper_test.aggregate( [
    s2orc_doi_is_string,
    s2orc_to_oa_doi_lookup,
    only_keep_existing_matches
] ))

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(len(res))

Process finished --- 28.988704204559326 seconds ---
92287


In [13]:
start_time = time.time()

oa_doi_is_string = { "$match": { "doi": { "$type": "string" }} }

oa_to_s2orc_doi_lookup = {
      "$lookup": {
         "from": "paper_test",
         "localField": "doi",
         "foreignField": "doi",
         "as": "matches"
      }
   }

res = list(db.works_oa_test.aggregate( [
    oa_doi_is_string,
    oa_to_s2orc_doi_lookup,
    only_keep_existing_matches
] ))

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(len(res))

Process finished --- 30.196167707443237 seconds ---
91704


In [14]:
start_time = time.time()

concise_s2orc_to_oa_doi_lookup = {
      "$lookup": {
         "from": "works_oa_test",
         "localField": "doi",
         "foreignField": "doi",
         "let": { "s2orc_doi": "$doi" },
         "pipeline": [ {
            "$match": {
               "$expr": { "$eq": [ "$$s2orc_doi", "$doi" ] }
            }
         } ],
         "as": "matches"
      }
   }

res = list(db.paper_test.aggregate( [
    s2orc_doi_is_string,
    concise_s2orc_to_oa_doi_lookup,
    only_keep_existing_matches
] )) 

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

Process finished --- 33.105173110961914 seconds ---
# hit: 92287


In [15]:
start_time = time.time()

oa_doi_is_string_pipeline = [ {
    "$match": {
        "doi": { "$type": "string" } ,
        "$expr": { "$eq": [ "$$s2orc_doi", "$doi" ] }
        }
    } ]

concise_s2orc_to_oa_doi_lookup = {
      "$lookup": {
         "from": "works_oa_test",
         "localField": "doi",
         "foreignField": "doi",
         "let": { "s2orc_doi": "$doi" },
         "pipeline": oa_doi_is_string_pipeline,
         "as": "matches"
      }
   }

res = list(db.paper_test.aggregate( [
    s2orc_doi_is_string,
    concise_s2orc_to_oa_doi_lookup,
    only_keep_existing_matches
] )) 

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

Process finished --- 35.64953589439392 seconds ---
# hit: 92287


In [16]:
db.works_oa_test.create_index([("concepts.display_name", ASCENDING)])

'concepts.display_name_1'

In [17]:
start_time = time.time()

oa_filter = { '$match': { 'concepts.display_name': 'Biology',  'doi': { "$type" : "string"} } }

res = list(db.works_oa_test.aggregate( [
    oa_filter,
    oa_to_s2orc_doi_lookup,
    only_keep_existing_matches
] ))

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

Process finished --- 6.486907482147217 seconds ---
# hit: 14085


In [18]:
db.works_oa_test.drop_index([("concepts.display_name", ASCENDING)])
db.works_oa_test.create_index([("concepts.display_name", ASCENDING)], partialFilterExpression = { "concepts.level" : 0})

'concepts.display_name_1'

In [19]:
start_time = time.time()

res = list(db.works_oa_test.aggregate( [
    oa_filter,
    oa_to_s2orc_doi_lookup,
    only_keep_existing_matches
] ))

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

Process finished --- 7.074239492416382 seconds ---
# hit: 14085


In [20]:
db.works_oa_test.drop_index([("concepts.display_name", ASCENDING)])
db.works_oa_test.create_index([("concepts.display_name", ASCENDING)])

'concepts.display_name_1'

In [21]:
db.paper_test.create_index([("s2fieldsofstudy.category", ASCENDING)])

's2fieldsofstudy.category_1'

In [22]:
start_time = time.time()

s2orc_filter = { '$match': { 's2fieldsofstudy.category': 'Biology', 'doi': { "$type" : "string"}  } }

res = list(db.paper_test.aggregate( [
    s2orc_filter,
    s2orc_to_oa_doi_lookup,
    only_keep_existing_matches
] ))

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

Process finished --- 4.731422185897827 seconds ---
# hit: 9003


In [23]:
start_time = time.time()

oa_filter_pipeline = [ {
    "$match": { 
        "concepts.display_name": "Biology",  
        "$expr":  {
            "$eq": [ "$$s2orc_doi", "$doi" ]
            }
        }
    } ]

concise_s2orc_to_oa_doi_lookup_concise = {
      "$lookup": {
         "from": "works_oa_test",
         "localField": "doi",
         "foreignField": "doi",
         "let": { "s2orc_doi": "$doi" },
         "pipeline": oa_filter_pipeline,
         "as": "matches"
      }
   }

res = list(db.paper_test.aggregate( [
   s2orc_filter,
   concise_s2orc_to_oa_doi_lookup_concise,
   only_keep_existing_matches
] ))

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

Process finished --- 4.58369255065918 seconds ---
# hit: 6396


In [24]:
db.works_oa_test.drop_index([("doi", ASCENDING)])
db.works_oa_test.drop_index([("concepts.display_name", ASCENDING)])
db.works_oa_test.create_index([("concepts.display_name", ASCENDING), ("doi", ASCENDING)])

db.paper_test.drop_index([("doi", ASCENDING)])
db.paper_test.drop_index([("s2fieldsofstudy.category", ASCENDING)])
db.paper_test.create_index([("s2fieldsofstudy.category", ASCENDING), ("doi", ASCENDING)])

's2fieldsofstudy.category_1_doi_1'

In [25]:
start_time = time.time()

res = list(db.paper_test.aggregate( [
   s2orc_filter,
   concise_s2orc_to_oa_doi_lookup_concise,
   only_keep_existing_matches
] ))

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

Process finished --- 4.7239556312561035 seconds ---
# hit: 6396


In [26]:
db.works_oa_test.drop_index([("concepts.display_name", ASCENDING), ("doi", ASCENDING)])
db.works_oa_test.create_index([("concepts.display_name", ASCENDING)])
db.works_oa_test.create_index([("doi", ASCENDING)])

db.paper_test.drop_index([("s2fieldsofstudy.category", ASCENDING), ("doi", ASCENDING)])
db.paper_test.create_index([("s2fieldsofstudy.category", ASCENDING)])
db.paper_test.create_index([("doi", ASCENDING)])

'doi_1'

In [27]:
db.paper_test.find_one({ "doi":  None, "externalids.MAG": { "$ne": None } })

{'_id': ObjectId('63e50688a64b9c3ca0a79ffd'),
 'externalids': {'ACL': None,
  'DBLP': None,
  'ArXiv': None,
  'MAG': '580729734',
  'CorpusId': '231379768',
  'PubMed': None,
  'DOI': None,
  'PubMedCentral': None},
 's2fieldsofstudy': None,
 'oa_works': True,
 'doi': None}

In [28]:
db.works_oa_test.find_one({ "doi":  None, "ids.mag": { "$ne": None } }, {"ids": 1, "doi": 1})

{'_id': ObjectId('6420450f8238317efbef948c'),
 'doi': None,
 'ids': {'openalex': 'https://openalex.org/W2297889166', 'mag': 2297889166}}

In [29]:
db.works_oa_test.update_many(
    {},
    [
        { "$addFields": { "mag": { "$toString": "$ids.mag" } } }
    ]
)

db.works_oa_test.create_index([("mag", ASCENDING)])
db.paper_test.create_index([("externalids.MAG", ASCENDING)])

# helpers
oa_mag_is_string = { "$match": { "mag": { "$type": "string" }  } }

In [30]:
start_time = time.time()

s2orc_paper_filter = {
            "$match": { 
               "doi": { "$type": "null" }, 
               "$expr":  { 
                    "$eq": [ "$$works_oa_mag", "$externalids.MAG" ] 
                }
            }
         }

works_oa2paper_lookup_concise = {
      "$lookup": {
         "from": "paper_test",
         "localField": "mag",
         "foreignField": "externalids.MAG",
         "let": { "works_oa_mag": "$mag" },
         "pipeline": [ s2orc_paper_filter ],
         "as": "matches"
      }
   }

oa_bio_filter_mag_is_string = {"$match": {"concepts.display_name": "Biology", "mag": { "$type": "string" }}}

res = list(db.works_oa_test.aggregate( [
    oa_bio_filter_mag_is_string,
    works_oa2paper_lookup_concise,
    only_keep_existing_matches
] ))

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

Process finished --- 16.91268515586853 seconds ---
# hit: 43754


In [31]:
start_time = time.time()

works_s2orc2oa_lookup = {
      "$lookup": {
         "from": "works_oa_test",
         "localField": "externalids.MAG",
         "foreignField": "mag",
         "as": "matches"
      }
   }

s2orc_bio_filter_mag_is_string = { "$match": {"s2fieldsofstudy.category": "Biology", "externalids.MAG": { "$type": "string" }, "doi": { "$type": "null" } } }

res = list(db.paper_test.aggregate( [
    s2orc_bio_filter_mag_is_string,
    works_s2orc2oa_lookup,
    only_keep_existing_matches
] ))

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

Process finished --- 9.05085802078247 seconds ---
# hit: 26356


In [32]:
start_time = time.time()

oa_paper_filter = {
            "$match": { 
               "doi": { "$type": "null" }, 
               "$expr":  { 
                    "$eq": [ "$$s2orc_mag", "$mag" ] 
                }
            }
         }

works_s2orc2oa_lookup_concise = {
      "$lookup": {
         "from": "works_oa_test",
         "localField": "externalids.MAG",
         "foreignField": "mag",
         "let": { "s2orc_mag": "$externalids.MAG" },
         "pipeline": [ oa_paper_filter ],
         "as": "matches"
      }
   }

s2orc_bio_filter_mag_is_string = { "$match": {"s2fieldsofstudy.category": "Biology", "externalids.MAG": { "$type": "string" }, "doi": { "$type": "null" } } }

res = list(db.paper_test.aggregate( [
    s2orc_bio_filter_mag_is_string,
    works_s2orc2oa_lookup,
    only_keep_existing_matches
] ))

print("Process finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

Process finished --- 8.847830295562744 seconds ---
# hit: 26356


In [33]:
start_time = time.time()

s2orc_bio_filter_doi_is_string = { '$match': { 's2fieldsofstudy.category': 'Biology', 'doi': { "$type" : "string"}  } }

concise_s2orc_to_oa_doi_lookup = {
      "$lookup": {
         "from": "works_oa_test",
         "localField": "doi",
         "foreignField": "doi",
         "as": "matches"
      }
   }

res = list(db.paper_test.aggregate( [
   s2orc_bio_filter_doi_is_string,
   concise_s2orc_to_oa_doi_lookup,
   only_keep_existing_matches,
   { '$addFields': { 'works_oa': {'$cond': [{'$ne': ['$matches', []]}, "$matches", None]} } },
   { '$project': { 'matches': 0 } }
] ))

print("Aggregation pipeline finished --- %s seconds ---" % (time.time() - start_time))

db.paper_test.bulk_write([
    UpdateOne( { 'doi': doc['doi'] }, { '$set': {'works_oa': doc['works_oa'][0]} })
    for doc in res
])

print("Process finished --- %s seconds ---" % (time.time() - start_time))

Aggregation pipeline finished --- 3.0302441120147705 seconds ---


Process finished --- 4.871156454086304 seconds ---


In [34]:
start_time = time.time()

s2orc_bio_filter_mag_is_string = { "$match": {"s2fieldsofstudy.category": "Biology", "externalids.MAG": { "$type": "string" }, "doi": { "$type": "null" } } }

concise_s2orc_to_oa_mag_lookup = {
      "$lookup": {
         "from": "works_oa_test",
         "localField": "externalids.MAG",
         "foreignField": "mag",
         "as": "matches"
      }
   }

res = list(db.paper_test.aggregate( [
   s2orc_bio_filter_mag_is_string,
   concise_s2orc_to_oa_mag_lookup,
   only_keep_existing_matches,
   { '$addFields': { 'works_oa': {'$cond': [{'$ne': ['$matches', []]}, "$matches", None]} } },
   { '$project': { 'matches': 0 } }
] ))

print("Aggregation pipeline finished --- %s seconds ---" % (time.time() - start_time))
print(f"# hit: {len(res)}")

db.paper_test.bulk_write([ 
    UpdateOne( {'externalids.MAG': doc['externalids']["MAG"]}, {'$set': {'works_oa': doc['works_oa'][0]}} ) 
    for doc in res
    ])

print("Process finished --- %s seconds ---" % (time.time() - start_time))

Aggregation pipeline finished --- 5.860445976257324 seconds ---
# hit: 26356


Process finished --- 10.8723726272583 seconds ---


In [35]:
db.paper_test.find_one({'externalids.MAG':  res[0]['externalids']['MAG']})

{'_id': ObjectId('63e50688a64b9c3ca0a7a375'),
 'externalids': {'ACL': None,
  'DBLP': None,
  'ArXiv': None,
  'MAG': '2462552499',
  'CorpusId': '46587026',
  'PubMed': '13854036',
  'DOI': None,
  'PubMedCentral': None},
 's2fieldsofstudy': [{'category': 'Biology', 'source': 's2-fos-model'},
  {'category': 'Chemistry', 'source': 'external'},
  {'category': 'Medicine', 'source': 'external'}],
 'oa_works': True,
 'doi': None,
 'works_oa': {'_id': ObjectId('64227602249c314cafa2ee23'),
  'doi': None,
  'publication_date': '1960-03-01',
  'ids': {'openalex': 'https://openalex.org/W2462552499',
   'pmid': 'https://pubmed.ncbi.nlm.nih.gov/13854036',
   'mag': 2462552499},
  'authorships': [{'institutions': []},
   {'institutions': []},
   {'institutions': []}],
  'concepts': [{'id': 'https://openalex.org/C2779317461',
    'wikidata': 'https://www.wikidata.org/wiki/Q16986094',
    'display_name': 'Mycobacterium phlei',
    'level': 4,
    'score': 0.97717744},
   {'id': 'https://openalex.org